## BERT multilingual sentence embeddings

For this we will be using the [bert-as-a-service](https://github.com/hanxiao/bert-as-service) module which starts a server to get BERT sentence embeddings of a sentence. To get this the library uses various pooling strategies over tokens. 

First install bert-serving-server and bert-serving-client by uncommenting the lines below. Note that this requires  1.10 <= PyTorch version <= 1.14 and the server needs to be Python3. The client can be either Py2 or Py3.

Install [PyTorch](https://pytorch.org/get-started/locally/) for your required OS first.

To know more about these sentence vectors are encoded, read [this](https://github.com/hanxiao/bert-as-service#q-what-are-the-available-pooling-strategies)


In [ ]:
# !pip install bert-serving-server
# !pip install bert-serving-client

# Now download one of the bert models and run the bert server. Here I load the bert-base-cased
# Note that this server needs TF 1.10<= version < 2.0, so I started the server on another virtualenv 
# !bert-serving-start -model_dir ~/Downloads/multi_cased_L-12_H-768_A-12 -num_worker=4

In [8]:
import os
os.getcwd()

'/Users/skhurana/pycon'

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from bert_serving.client import BertClient
bc = BertClient()

Now we'll encode sentences in different languages and try to see how their embeddings differ.

In [15]:
sentences = bc.encode(["Two dogs are wrestling and hugging", 
                       "There is no dog wrestling and hugging",
                       "दो कुत्ते कुश्ती और गले मिल रहे हैं",
                       "कुश्ती और गले लगने वाले कुत्ते नहीं हैं", 
                       "Tengo muchas ganas de ir a la escuela"
                      ])

In [9]:
sentences.shape

(5, 768)

In [16]:
cosine_similarity(sentences[0][:].reshape(1,-1),sentences[2][:].reshape(1,-1))

array([[0.85172975]], dtype=float32)

In [17]:
cosine_similarity(sentences[1][:].reshape(1,-1),sentences[3][:].reshape(1,-1))

array([[0.8633849]], dtype=float32)

In [19]:
cosine_similarity(sentences[0][:].reshape(1,-1),sentences[4][:].reshape(1,-1))

array([[0.8814724]], dtype=float32)

### Tokenizer

Now we'll be using the [Huggingface library](https://github.com/huggingface/transformers) for the tokenizer and transformers. To see the installation part, see point 3 in [this](https://github.com/ShreyaKhurana/wwc/blob/master/README.md)

In [13]:
import torch

In [3]:
from tokenizers import *
from transformers import *

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [5]:
tokenizer.tokenize("Hola quieres ir al centro comercial hoy")

['Ho', '##la', 'quiere', '##s', 'ir', 'al', 'centro', 'comercial', 'hoy']

In [6]:
tokenizer.tokenize("चलो आज कहीं चलते हैं")

['च', '##ल', '##ो', 'आज', 'क', '##ही', '##ं', 'च', '##ल', '##ते', 'हैं']

In [8]:
tokenizer.tokenize("Bears. Beets. Battlestar Gallactica.")

['Bears',
 '.',
 'Bee',
 '##ts',
 '.',
 'Battle',
 '##star',
 'Gall',
 '##act',
 '##ica',
 '.']

### Masked LM

Let's say you want to evaluate how the language model is performing over your multilingual corpus, which might have code-switched data like the example shown below.

In [9]:
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [11]:
# Modified from https://mayhewsw.github.io/2019/01/16/can-bert-generate-text/
def predict_word(text, target):
    
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index(target)
    tokenized_text[masked_index] = '[MASK]'

    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Predict all tokens
    predictions = model(tokens_tensor, segments_tensors)
    predicted_index = torch.argmax(predictions[0][0, masked_index, :]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])

    print("Original:", text)
    print("Masked:", " ".join(tokenized_text))
    print("Predicted token:", predicted_token)
    print("Other options:")
    
    for i in range(10):
        predictions[0][0,masked_index,predicted_index] = -11100000
        predicted_index = torch.argmax(predictions[0][0, masked_index]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
        print(predicted_token)

In [10]:
text = """
This is what I want to say to everyone out there - vivir y dejar vivir!
"""

tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

['This', 'is', 'what', 'I', 'want', 'to', 'say', 'to', 'everyone', 'out', 'there', '-', 'vivir', 'y', 'dejar', 'vivir', '!']


In [14]:
predict_word(text, "vivir")

Original: 
This is what I want to say to everyone out there - vivir y dejar vivir!

Masked: This is what I want to say to everyone out there - [MASK] y dejar vivir !
Predicted token: ['¡']
Other options:
['...']
['Viva']
['Vive']
['Hasta']
['dejar']
['Dar']
['empezar']
['Gracias']
['volver']
['Por']


In [16]:
# ¡Viva!
predict_word(text, "dejar")

Original: 
This is what I want to say to everyone out there - vivir y dejar vivir!

Masked: This is what I want to say to everyone out there - vivir y [MASK] vivir !
Predicted token: ['no']
Other options:
['solo']
['non']
['bien']
['nunca']
['yo']
['a']
['sólo']
['también']
['hacer']
[',']


In [19]:
# Now let's see a code-switched example.
text = """
Indian moms at 8 am be like: "बेटा, उठो, 12 बज गए हैं"
"""
tokenizer.tokenize(text)

['Indian',
 'mo',
 '##ms',
 'at',
 '8',
 'am',
 'be',
 'like',
 ':',
 '"',
 'ब',
 '##ेट',
 '##ा',
 ',',
 'उ',
 '##ठ',
 '##ो',
 ',',
 '12',
 'ब',
 '##ज',
 'गए',
 'हैं',
 '"']

In [20]:
predict_word(text, "गए")

Original: 
Indian moms at 8 am be like: "बेटा, उठो, 12 बज गए हैं"

Masked: Indian mo ##ms at 8 am be like : " ब ##ेट ##ा , उ ##ठ ##ो , 12 ब ##ज [MASK] हैं "
Predicted token: ['##र']
Other options:
['##ते']
['##ट']
['##ू']
['##ूत']
['##ली']
['##ती']
['##ल']
['##ु']
['##ोर']
['##त']


In [37]:
# Let's see if the prediction is better for a monolingual simple example in Hindi
text = """
भारतीय माँ सुबह 8 बजे - बेटा, उठो, 12 बज गए हैं
"""


In [38]:
tokenizer.tokenize(text)

['भारतीय',
 'मा',
 '##ँ',
 'स',
 '##ु',
 '##ब',
 '##ह',
 '8',
 'ब',
 '##जे',
 '-',
 'ब',
 '##ेट',
 '##ा',
 ',',
 'उ',
 '##ठ',
 '##ो',
 ',',
 '12',
 'ब',
 '##ज',
 'गए',
 'हैं']

In [40]:
predict_word(text, "मा")

Original: 
भारतीय माँ सुबह 8 बजे - बेटा, उठो, 12 बज गए हैं

Masked: भारतीय [MASK] ##ँ स ##ु ##ब ##ह 8 ब ##जे - ब ##ेट ##ा , उ ##ठ ##ो , 12 ब ##ज गए हैं
Predicted token: ['मा']
Other options:
['आ']
['कहा']
['जा']
['अ']
['खा']
['हा']
['औ']
['का']
['बी']
['ऊ']


In [24]:
# Let's try a bigger piece of text
text = """
अगर आप हिंदी में जानकारी पढ़ना पसंद करते हैं तो आपको best hindi blogs यानी भारत में popular Hindi bloggers कौन-कौन से हैं इसकी जानकारी होनी चाहिए।

बहुत से लोगों को इंग्लिश में जानकारी होते हुए भी वो हिंदी में blog पढ़ना पसंद करते हैं लेकिन इसके लिए हमारे पास कुछ popular Hindi blogs list होने चाहिए।
"""
tokenizer.tokenize(text)

['अ',
 '##गर',
 'आ',
 '##प',
 'हिंदी',
 'में',
 'जानकारी',
 'प',
 '##ढ़',
 '##ना',
 'प',
 '##स',
 '##ंद',
 'करते',
 'हैं',
 'तो',
 'आ',
 '##प',
 '##को',
 'best',
 'hindi',
 'blog',
 '##s',
 'या',
 '##नी',
 'भारत',
 'में',
 'popular',
 'Hindi',
 'blog',
 '##gers',
 'क',
 '##ौ',
 '##न',
 '-',
 'क',
 '##ौ',
 '##न',
 'से',
 'हैं',
 'इसकी',
 'जानकारी',
 'हो',
 '##नी',
 'चाहिए',
 '।',
 'बहुत',
 'से',
 'लोगों',
 'को',
 'इंग्लिश',
 'में',
 'जानकारी',
 'होते',
 'हुए',
 'भी',
 'व',
 '##ो',
 'हिंदी',
 'में',
 'blog',
 'प',
 '##ढ़',
 '##ना',
 'प',
 '##स',
 '##ंद',
 'करते',
 'हैं',
 'लेकिन',
 'इसके',
 'लिए',
 'हम',
 '##ारे',
 'पास',
 'कुछ',
 'popular',
 'Hindi',
 'blog',
 '##s',
 'list',
 'होने',
 'चाहिए',
 '।']

In [25]:
# It was able to recognize the mased word as its predicted first choice! But notice how useful the bidirectionality is here.
predict_word(text, "blog")

Original: 
अगर आप हिंदी में जानकारी पढ़ना पसंद करते हैं तो आपको best hindi blogs यानी भारत में popular Hindi bloggers कौन-कौन से हैं इसकी जानकारी होनी चाहिए।

बहुत से लोगों को इंग्लिश में जानकारी होते हुए भी वो हिंदी में blog पढ़ना पसंद करते हैं लेकिन इसके लिए हमारे पास कुछ popular Hindi blogs list होने चाहिए।

Masked: अ ##गर आ ##प हिंदी में जानकारी प ##ढ़ ##ना प ##स ##ंद करते हैं तो आ ##प ##को best hindi [MASK] ##s या ##नी भारत में popular Hindi blog ##gers क ##ौ ##न - क ##ौ ##न से हैं इसकी जानकारी हो ##नी चाहिए । बहुत से लोगों को इंग्लिश में जानकारी होते हुए भी व ##ो हिंदी में blog प ##ढ़ ##ना प ##स ##ंद करते हैं लेकिन इसके लिए हम ##ारे पास कुछ popular Hindi blog ##s list होने चाहिए ।
Predicted token: ['blog']
Other options:
['Blog']
['publisher']
['forum']
['tag']
['poster']
['photographer']
['peer']
['reporter']
['guru']
['pseudonym']


In [27]:
# Let's try another word here
predict_word(text, "hindi")

Original: 
अगर आप हिंदी में जानकारी पढ़ना पसंद करते हैं तो आपको best hindi blogs यानी भारत में popular Hindi bloggers कौन-कौन से हैं इसकी जानकारी होनी चाहिए।

बहुत से लोगों को इंग्लिश में जानकारी होते हुए भी वो हिंदी में blog पढ़ना पसंद करते हैं लेकिन इसके लिए हमारे पास कुछ popular Hindi blogs list होने चाहिए।

Masked: अ ##गर आ ##प हिंदी में जानकारी प ##ढ़ ##ना प ##स ##ंद करते हैं तो आ ##प ##को best [MASK] blog ##s या ##नी भारत में popular Hindi blog ##gers क ##ौ ##न - क ##ौ ##न से हैं इसकी जानकारी हो ##नी चाहिए । बहुत से लोगों को इंग्लिश में जानकारी होते हुए भी व ##ो हिंदी में blog प ##ढ़ ##ना प ##स ##ंद करते हैं लेकिन इसके लिए हम ##ारे पास कुछ popular Hindi blog ##s list होने चाहिए ।
Predicted token: ['Hindi']
Other options:
['Indian']
['English']
['India']
['Punjabi']
['Bollywood']
['of']
['hindi']
['known']
['popular']
['Pakistani']


In [28]:
# Let's try another word here
predict_word(text, "इंग्लिश")

Original: 
अगर आप हिंदी में जानकारी पढ़ना पसंद करते हैं तो आपको best hindi blogs यानी भारत में popular Hindi bloggers कौन-कौन से हैं इसकी जानकारी होनी चाहिए।

बहुत से लोगों को इंग्लिश में जानकारी होते हुए भी वो हिंदी में blog पढ़ना पसंद करते हैं लेकिन इसके लिए हमारे पास कुछ popular Hindi blogs list होने चाहिए।

Masked: अ ##गर आ ##प हिंदी में जानकारी प ##ढ़ ##ना प ##स ##ंद करते हैं तो आ ##प ##को best hindi blog ##s या ##नी भारत में popular Hindi blog ##gers क ##ौ ##न - क ##ौ ##न से हैं इसकी जानकारी हो ##नी चाहिए । बहुत से लोगों को [MASK] में जानकारी होते हुए भी व ##ो हिंदी में blog प ##ढ़ ##ना प ##स ##ंद करते हैं लेकिन इसके लिए हम ##ारे पास कुछ popular Hindi blog ##s list होने चाहिए ।
Predicted token: ['हिंदी']
Other options:
['हिन्दी']
['अंग्रेजी']
['भारत']
['अंग्रेज़ी']
['मराठी']
['उर्दू']
['Hindi']
['भाषा']
['गुजराती']
['इंग्रजी']


In [29]:
predict_word(text, "list")

Original: 
अगर आप हिंदी में जानकारी पढ़ना पसंद करते हैं तो आपको best hindi blogs यानी भारत में popular Hindi bloggers कौन-कौन से हैं इसकी जानकारी होनी चाहिए।

बहुत से लोगों को इंग्लिश में जानकारी होते हुए भी वो हिंदी में blog पढ़ना पसंद करते हैं लेकिन इसके लिए हमारे पास कुछ popular Hindi blogs list होने चाहिए।

Masked: अ ##गर आ ##प हिंदी में जानकारी प ##ढ़ ##ना प ##स ##ंद करते हैं तो आ ##प ##को best hindi blog ##s या ##नी भारत में popular Hindi blog ##gers क ##ौ ##न - क ##ौ ##न से हैं इसकी जानकारी हो ##नी चाहिए । बहुत से लोगों को इंग्लिश में जानकारी होते हुए भी व ##ो हिंदी में blog प ##ढ़ ##ना प ##स ##ंद करते हैं लेकिन इसके लिए हम ##ारे पास कुछ popular Hindi blog ##s [MASK] होने चाहिए ।
Predicted token: ['भी']
Other options:
['नहीं']
['उपलब्ध']
['तैयार']
['प्राप्त']
['प्रकाशित']
['ही']
['प्रयोग']
['न']
['कम']
['शामिल']


### Transliterated data


Note that BERT has NOT been trained for this, it only has had access to multiple languages in their own script -> which is why we see pretty decent results above. But because it has been trained for languages in Roman scripts like English, Spanish, German etc. we might see some results, we would expect them to be gibberish.

In [34]:
# A common place where you will find transliterated data is lyrics and thanks to Bollywood, Hindi has plenty!!
text = """Aasma Hai Neela Kyun, Paani Geela Geela Kyun
Gol Kyun Hai Zameen, Silk Mein Hai Narmi Kyun
Aag Mein Hai Garmi Kyun, Do Aur Do Paanch Kyun Nahi
Pedd Ho Gaye Kam Kyun, Teen Hain Ye Mausam Kyun
Chand Do Kyun Nahi, Duniya Mein Hai Jung Kyun
Behta Laal Rang Kyun, Sarhadein Hain Kyun Har Kahin
Socha Hai... Yeh Tumne Kya Kabhi
Socha Hai... Ki Hain Yeh Kya Sabhi Socha Hai
Socha Nahi To Socho Abhi.....
Behti Kyun Hai Har Nadi, Hoti Kya Hai Roshni
Barf Girti Hai Kyun, Dost Kyun Hain Roothte
Taare Kyun Hain Tuttey, Baadlon Mein Bijli Hai Kyun
Socha Hai... Yeh Tumne Kya Kabhi
Socha Hai... Ki Hain Yeh Kya Sabhi Socha Hai
Socha Nahi To Socho Abhi
Sannata Sunai Nahin Deta, Aur Hawayen Dikhayi Nahin Deti
Socha Hai Kya Kabhi, Hota Hai Yeh Kyun
"""

tokenizer.tokenize(text)

['Aa',
 '##sma',
 'Hai',
 'Ne',
 '##ela',
 'Ky',
 '##un',
 ',',
 'Pa',
 '##ani',
 'G',
 '##eel',
 '##a',
 'G',
 '##eel',
 '##a',
 'Ky',
 '##un',
 'Gol',
 'Ky',
 '##un',
 'Hai',
 'Za',
 '##meen',
 ',',
 'Silk',
 'Mein',
 'Hai',
 'Na',
 '##rmi',
 'Ky',
 '##un',
 'Aa',
 '##g',
 'Mein',
 'Hai',
 'Ga',
 '##rmi',
 'Ky',
 '##un',
 ',',
 'Do',
 'Aur',
 'Do',
 'Pa',
 '##anc',
 '##h',
 'Ky',
 '##un',
 'Na',
 '##hi',
 'Pe',
 '##dd',
 'Ho',
 'Gay',
 '##e',
 'Ka',
 '##m',
 'Ky',
 '##un',
 ',',
 'Teen',
 'Hai',
 '##n',
 'Ye',
 'Mau',
 '##sam',
 'Ky',
 '##un',
 'Chan',
 '##d',
 'Do',
 'Ky',
 '##un',
 'Na',
 '##hi',
 ',',
 'Du',
 '##niya',
 'Mein',
 'Hai',
 'Jung',
 'Ky',
 '##un',
 'Be',
 '##hta',
 'La',
 '##al',
 'Rang',
 'Ky',
 '##un',
 ',',
 'Sar',
 '##had',
 '##ein',
 'Hai',
 '##n',
 'Ky',
 '##un',
 'Har',
 'Ka',
 '##hin',
 'Soc',
 '##ha',
 'Hai',
 '.',
 '.',
 '.',
 'Ye',
 '##h',
 'Tu',
 '##mne',
 'Ky',
 '##a',
 'Ka',
 '##bh',
 '##i',
 'Soc',
 '##ha',
 'Hai',
 '.',
 '.',
 '.',
 'Ki',
 'Hai',
 '##n

In [32]:
predict_word(text, "Gol")

Original: Aasma Hai Neela Kyun, Paani Geela Geela Kyun
Gol Kyun Hai Zameen, Silk Mein Hai Narmi Kyun
Aag Mein Hai Garmi Kyun, Do Aur Do Paanch Kyun Nahi
Pedd Ho Gaye Kam Kyun, Teen Hain Ye Mausam Kyun
Chand Do Kyun Nahi, Duniya Mein Hai Jung Kyun
Behta Laal Rang Kyun, Sarhadein Hain Kyun Har Kahin
Socha Hai... Yeh Tumne Kya Kabhi
Socha Hai... Ki Hain Yeh Kya Sabhi Socha Hai
Socha Nahi To Socho Abhi.....
Behti Kyun Hai Har Nadi, Hoti Kya Hai Roshni
Barf Girti Hai Kyun, Dost Kyun Hain Roothte
Taare Kyun Hain Tuttey, Baadlon Mein Bijli Hai Kyun
Socha Hai... Yeh Tumne Kya Kabhi
Socha Hai... Ki Hain Yeh Kya Sabhi Socha Hai
Socha Nahi To Socho Abhi
Sannata Sunai Nahin Deta, Aur Hawayen Dikhayi Nahin Deti
Socha Hai Kya Kabhi, Hota Hai Yeh Kyun

Masked: Aa ##sma Hai Ne ##ela Ky ##un , Pa ##ani G ##eel ##a G ##eel ##a Ky ##un [MASK] Ky ##un Hai Za ##meen , Silk Mein Hai Na ##rmi Ky ##un Aa ##g Mein Hai Ga ##rmi Ky ##un , Do Aur Do Pa ##anc ##h Ky ##un Na ##hi Pe ##dd Ho Gay ##e Ka ##m Ky ##un ,

In [35]:
predict_word(text, "Mein")
# Notice WHY it was able to correctly predict it -> it has a couple of other similar contexts to compare to.

Original: Aasma Hai Neela Kyun, Paani Geela Geela Kyun
Gol Kyun Hai Zameen, Silk Mein Hai Narmi Kyun
Aag Mein Hai Garmi Kyun, Do Aur Do Paanch Kyun Nahi
Pedd Ho Gaye Kam Kyun, Teen Hain Ye Mausam Kyun
Chand Do Kyun Nahi, Duniya Mein Hai Jung Kyun
Behta Laal Rang Kyun, Sarhadein Hain Kyun Har Kahin
Socha Hai... Yeh Tumne Kya Kabhi
Socha Hai... Ki Hain Yeh Kya Sabhi Socha Hai
Socha Nahi To Socho Abhi.....
Behti Kyun Hai Har Nadi, Hoti Kya Hai Roshni
Barf Girti Hai Kyun, Dost Kyun Hain Roothte
Taare Kyun Hain Tuttey, Baadlon Mein Bijli Hai Kyun
Socha Hai... Yeh Tumne Kya Kabhi
Socha Hai... Ki Hain Yeh Kya Sabhi Socha Hai
Socha Nahi To Socho Abhi
Sannata Sunai Nahin Deta, Aur Hawayen Dikhayi Nahin Deti
Socha Hai Kya Kabhi, Hota Hai Yeh Kyun

Masked: Aa ##sma Hai Ne ##ela Ky ##un , Pa ##ani G ##eel ##a G ##eel ##a Ky ##un Gol Ky ##un Hai Za ##meen , Silk [MASK] Hai Na ##rmi Ky ##un Aa ##g Mein Hai Ga ##rmi Ky ##un , Do Aur Do Pa ##anc ##h Ky ##un Na ##hi Pe ##dd Ho Gay ##e Ka ##m Ky ##un , 